# 01 - Advanced Feature Engineering

This notebook applies business metrics and temporal feature transformations to prepare data for ML modeling.

**Course:** DATA 230 (Data Visualization) at SJSU


In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from src.features.business_metrics import BusinessMetrics
from src.features.temporal_features import TemporalFeatures

# Load cleaned data
df = pd.read_csv('../data/processed/cleaned_data.csv')
print(f"Loaded {len(df)} records")
